In [2]:
!pip install datasets

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import datasets
import os
import random
import torchmetrics

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk

from collections import Counter


os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
random.seed(42)

torch.manual_seed(42) 
torch.cuda.manual_seed(42)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Data Preparation


In [6]:
dataset = datasets.load_dataset('ag_news')

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [7]:
words = Counter()

for example in tqdm(dataset['train']['text']):
    prccessed_text = example.lower().translate(
        str.maketrans('', '', string.punctuation))

    for word in word_tokenize(prccessed_text):
        words[word] += 1


vocab = set(['<unk>', '<pad>'])
counter_threshold = 25

for char, cnt in words.items():
    if cnt > counter_threshold:
        vocab.add(char)

print(f'Vocab size: {len(vocab)}')

word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

100%|██████████| 120000/120000 [00:49<00:00, 2411.83it/s]

Vocab size: 11840


In [8]:
class NewsDataset:
    def __init__(self, sentences):
        self.data = sentences
        self.unk_id = word2ind['<unk>']
        self.pad_id = word2ind['<pad>']

    def __getitem__(self, idx):
        processed_text = self.data[idx]['text'].lower().translate(
            str.maketrans('', '', string.punctuation))
        tokenized_sentence = []
        tokenized_sentence += [
            word2ind.get(word, self.unk_id) for word in word_tokenize(processed_text)
            ] 

        train_sample = {
            "text": torch.LongTensor(tokenized_sentence),
            "label": torch.LongTensor([self.data[idx]['label']])
        }

        return train_sample

    def __len__(self):
        return len(self.data)


In [9]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    sequences = [item['text'] for item in batch]
    targets = [item['label'] for item in batch]
    
    MAX_LENGTH = 512
    sequences = [seq[:MAX_LENGTH] for seq in sequences]
    
    sequences_padded = pad_sequence(sequences=sequences, batch_first=True, padding_value=word2ind['<pad>'])
    
    return sequences_padded, torch.LongTensor(targets)

In [10]:
train_dataset = NewsDataset(dataset['train'])

np.random.seed(42)
idx = np.random.choice(np.arange(len(dataset['test'])), 5000)
eval_dataset = NewsDataset(dataset['test'].select(idx))

batch_size = 128
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=collate_fn, batch_size=batch_size , num_workers=os.cpu_count(), pin_memory=True)

eval_dataloader = DataLoader(
    eval_dataset, shuffle=False, collate_fn=collate_fn, batch_size=batch_size, num_workers=os.cpu_count(), pin_memory=True)

# Model

In [11]:
class RNN(nn.Module):
    def __init__( self,  vocab_size, hidden_dim=256, embed_dim=512, num_classes = 4):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=word2ind['<pad>'])
        self.rnn = nn.LSTM(input_size=embed_dim, 
                           hidden_size=hidden_dim, 
                           num_layers=3, 
                           batch_first=True,
                           bidirectional=True,
                           dropout=0.2)
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, num_classes)
        )
        

    def forward(self, input_batch):
        embeddings = self.embedding(input_batch) 
        output, (_, _) = self.rnn(embeddings) 
        output = output.mean(dim=1)

        return self.fc(output)

In [12]:
model = RNN(vocab_size=len(vocab)).to(device)
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)
optim = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
amp_scaler = torch.cuda.amp.GradScaler()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=3, factor=0.5)

accuracy = torchmetrics.classification.MulticlassAccuracy(4).to(device)

In [13]:
for epoch in range(300):
    
    train_total_loss = 0
    train_total_acc = 0
    model.train()
    loop = tqdm(train_dataloader)
    for X,y in loop:
        X,y = X.to(device), y.to(device)
        with torch.cuda.amp.autocast():
            logits = model(X)
            loss = loss_fn(logits, y)
        
        optim.zero_grad(set_to_none='True')
        amp_scaler.scale(loss).backward()
        amp_scaler.step(optim)
        amp_scaler.update()
        
        train_total_loss += loss
        train_total_acc += accuracy(logits, y)
        
    train_total_loss /= len(train_dataloader)
    train_total_acc /= len(train_dataloader)
    
    
    test_total_loss = 0
    test_total_acc = 0
    model.eval()
    with torch.inference_mode():
        loop = tqdm(eval_dataloader)
        for X,y in loop:
            X,y = X.to(device), y.to(device)
            logits = model(X)
            loss = loss_fn(logits, y)
            
            test_total_loss += loss
            test_total_acc += accuracy(logits, y)
            
        test_total_loss /= len(eval_dataloader)
        test_total_acc /= len(eval_dataloader)
    
    scheduler.step(test_total_acc)
    
    print(f"epoch: {epoch}   train_loss: {train_total_loss}, train_acc: {train_total_acc}, test_loss: {test_total_loss}, test_acc: {test_total_acc}")

100%|██████████| 40/40 [00:01<00:00, 23.43it/s]


epoch: 0   train_loss: 0.6439827680587769, train_acc: 0.8486865162849426, test_loss: 0.5729407668113708, test_acc: 0.8958719372749329


100%|██████████| 40/40 [00:02<00:00, 19.78it/s]


epoch: 1   train_loss: 0.5221606492996216, train_acc: 0.917437732219696, test_loss: 0.5527138113975525, test_acc: 0.9036030769348145


100%|██████████| 40/40 [00:01<00:00, 21.89it/s]


...

epoch: 17   train_loss: 0.3617021441459656, train_acc: 0.995691180229187, test_loss: 0.5765952467918396, test_acc: 0.9161124229431152


100%|██████████| 40/40 [00:01<00:00, 22.56it/s]


epoch: 18   train_loss: 0.36093997955322266, train_acc: 0.9960164427757263, test_loss: 0.5758065581321716, test_acc: 0.91604083776474


 42%|████▏     | 393/938 [00:34<00:48, 11.33it/s]


KeyboardInterrupt: 